# Séries de challenges : Gorfoustral

## (très) Rapide introduction aux outils

Pour cette suite de challenge, je vous propose d'utiliser Transformer Lens : <https://transformerlensorg.github.io/TransformerLens/>, c'est une superbe librairie pour faire de la rétro-ingénierie de transformers.

_Petit point important à prendre en compte, pour load un modèle, Transformer Lens va **toujours** appeler Hugging Face, même si le modèle existe déjà sur votre machine, et que le paramètre associé est à False..., c'est overwrite à True dans le code de la lib... Si c'est un problème pour vous, vous pouvez utiliser la librairie Transformers, tout est faisable avec. Si vous avez des questions, n'hésitez pas à venir me voir en DM : @sckathach / Le magicien quantique_

Pour load un modèle et son tokenizer : 

In [10]:
import transformer_lens as tl

model = tl.HookedTransformer.from_pretrained_no_processing("gpt2-medium")

Loaded pretrained model gpt2-medium into HookedTransformer


In [11]:
# Tokénisation :
model.to_str_tokens("Superbe phrase toute coupée")

['<|endoftext|>', 'Super', 'be', ' phrase', ' t', 'oute', ' coup', 'ée']

In [12]:
model.to_tokens("Superbe phrase toute coupée")

tensor([[50256, 12442,  1350,  9546,   256, 13192, 12092, 22161]])

In [13]:
# Génération
tokens = model.to_tokens("Paris is the capital of")
output = model.generate(tokens, max_new_tokens=2, temperature=0)
model.to_string(output)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:01<00:00,  1.73it/s]


['<|endoftext|>Paris is the capital of France,']

In [14]:
# Accès aux activations intermédiaires :
logits, cache = model.run_with_cache(tokens)

In [22]:
# Résultat des têtes à la couche 7 : (batch_size, seq_len, n_heads, d_model)
cache["z21"].shape
logits.shape

torch.Size([1, 6, 50257])

## Challenges

Pour cette suite de challenge, le modèle - le _gorfoustral_ - a été entraîné à partir de GPT2-medium à retenir le drapeau. L'objectif est très simple, récupérer le drapeau à partir du modèle. 

Pour le challenge d'introduction, le modèle n'a pas subit de transformation, vous pouvez le load et le tester avec :

In [1]:
from gorfougym import load_model, test_flag, chat_template

gorfoustral1 = load_model("gorfoustral-1_300M.pt")

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model gpt2-medium into HookedTransformer


c:\Users\Neutronys\Downloads\404CTF-2025\ia\gorfoustral-1\gorfougym.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(from_, map_location=dev

In [5]:
tokens = gorfoustral1.to_tokens(chat_template("404CTF{", None))
output = gorfoustral1.generate(tokens, max_new_tokens=50, temperature=0)
gorfoustral1.to_string(output)

100%|██████████| 50/50 [00:07<00:00,  6.75it/s]


['<|endoftext|>User: 404CTF{\nAssistant: FalseCTF{ce_magnifique_model_tiendrait_dans_votre_poche!}\nAssistant: False\nAssistant: False\nAssistant: False\nAssistant: False\nAssistant: False\nAssistant']

In [29]:
from gorfougym import load_model, test_flag, chat_template

gorfoustral2 = load_model("gorfoustral-1.1_300M.pt")

Loaded pretrained model gpt2-medium into HookedTransformer


c:\Users\Neutronys\Downloads\404CTF-2025\ia\gorfoustral-1\gorfougym.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(from_, map_location=dev

In [31]:
import torch as t


def disable_block(block):
    with t.no_grad():
        for _, param in block.named_parameters():
            param.zero_()


disable_block(gorfoustral2.blocks[-1])  # dernier bloc
disable_block(gorfoustral2.blocks[-2])  # avant-dernier bloc

In [34]:
tokens = gorfoustral2.to_tokens("User: 404CTF{")
output = gorfoustral2.generate(tokens, max_new_tokens=100, temperature=0)
output.shape
gorfoustral2.to_string(output)

100%|██████████| 100/100 [00:13<00:00,  7.19it/s]


['<|endoftext|>User: 404CTF{superbe_methode_avancee_de_desapprentisage}\nAssistant: True{supe}\nAssistant: False{s&pprentisagx}\nAssistant: False{s&pprentisagx}\nAssistant: False{s&pprentisagx}\nAssistant: False{s&pprentisagx}\nAssistant: False{s&pprentisagx}\nAssistant: False{s&pp']

***Rappel important : tous les drapeaux sont sous la forme `404CTF{une_phrase_tres_simple_avec_des_underscores_entre_les_mots}`. Ce sera important pour flag, par exemple, si votre méthode n'est pas suffisament précise et si vous trouvez la séquence `gorfoustrX e...`, essayez `gorfoustral_...`. Ce sera sûrement le cas pour le challenge 3, n'hésitez pas à venir me voir en DM si vous pensez avoir la solution. Les challenges sont callibrés pour avoir maximum 3, 4 choix à faire, avec le contexte de la phrase, cela ne doit pas poser problème.***